**CARREGAMENTO DOS DADOS**

In [11]:
from google.colab import files

# Esta linha irá abrir um seletor de arquivos no seu computador
uploaded = files.upload()

import pandas as pd

df = pd.read_csv('ecommerce_customer_data_large_desformatado.csv')

# Exibir as primeiras linhas do dataframe para verificar se o carregamento foi bem-sucedido
df.head()

Saving ecommerce_customer_data_large_desformatado.csv to ecommerce_customer_data_large_desformatado (1).csv


,Customer ID,Purchase Date,Product Category,Product Price,Quantity,Total Purchase Amount,Payment Method,Customer Age,Returns,Customer Name,Age,Gender,Churn
0,44605,2023-05-03 21:30:02,Home,177,1,2427,PayPal,31,1.0,John Rivera,31,Female,0
1,44605,2021-05-16 13:57:44,Electronics,174,3,2448,PayPal,31,1.0,John Rivera,31,Female,0
2,44605,2020-07-13 06:16:57,Books,413,1,2345,Credit Card,31,1.0,John Rivera,31,Female,0
3,44605,2023-01-17 13:14:36,Electronics,396,3,937,Cash,31,0.0,John Rivera,31,Female,0
4,44605,2021-05-01 11:29:27,Books,259,4,2598,PayPal,31,1.0,John Rivera,31,Female,0


**TRATAMENTO DOS DADOS**

Primeiro realizou-se a formatação da coluna 'Purchase Date' para o formato 'dia/mes/ano'. Em seguida, eliminou-se a coluna 'Age', que estava repetida. Trocou-se, também, os espaços vazios de palavras composta por "_". Por fim, os nulos foram trocados por zero.

In [12]:
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])

# Formatando a data para 'dia/mês/ano'
df['Purchase Date'] = df['Purchase Date'].dt.strftime('%d/%m/%Y')

# Exibir as primeiras linhas para verificar a mudança
print('Mudança Data')
print(df.head())

# Deletar a coluna 'Age'
df = df.drop(columns=['Age'])

# Exibir as primeiras linhas para verificar a remoção da coluna
print('Mudança Age')
print(df.head())

# Substituir todos os valores NaN por 0 em todo o DataFrame
df = df.fillna(0)

# Exibir as primeiras linhas para verificar a mudança
print('Mudança nulo')
print(df.head())

# Substituir espaços em branco por underscores nos nomes das colunas
df.columns = df.columns.str.replace(' ', '_')

#Exibir as primeiras linhas para verificar a mudança
print('Mudança Plavra Composta')
print(df.head())

Mudança Data
   Customer ID Purchase Date Product Category  Product Price  Quantity  \
0        44605    03/05/2023             Home            177         1   
1        44605    16/05/2021      Electronics            174         3   
2        44605    13/07/2020            Books            413         1   
3        44605    17/01/2023      Electronics            396         3   
4        44605    01/05/2021            Books            259         4   

   Total Purchase Amount Payment Method  Customer Age  Returns Customer Name  \
0                   2427         PayPal            31      1.0   John Rivera   
1                   2448         PayPal            31      1.0   John Rivera   
2                   2345    Credit Card            31      1.0   John Rivera   
3                    937           Cash            31      0.0   John Rivera   
4                   2598         PayPal            31      1.0   John Rivera   

   Age  Gender  Churn  
0   31  Female      0  
1   31  Femal

**Criação de Dimensões**


*   Time Dimension: Chave 'Purchase Date' com colunas 'dia', 'mes' e 'ano';
*   Customer Dimension: Chave 'Customer ID' com colunas 'Customer Age', 'Customer Name', 'Gender', 'Payment Method', 'Churn', e 'Returns';
*   Category Dimension: Chave 'Product Category ID' com colunas 'Product Price' e 'Product Category';
*   Mapear 'Product Category' para 'Product Category ID' na tabela principal;
*   Criação da Fato: 'Customer ID', 'Purchase Date', 'Product Category ID', 'Quantity' e 'Total Purchase Amount';







In [15]:
# Criar a Tabela de Dimensão do Tempo (Time Dimension)
df['Purchase_Date'] = pd.to_datetime(df['Purchase_Date'], format='%d/%m/%Y')
time_dimension = df[['Purchase_Date']].drop_duplicates().reset_index(drop=True)
time_dimension['Day'] = time_dimension['Purchase_Date'].dt.day
time_dimension['Month'] = time_dimension['Purchase_Date'].dt.month
time_dimension['Year'] = time_dimension['Purchase_Date'].dt.year

print("Tabela de Dimensão do Tempo (Time Dimension):")
print(time_dimension.head())

# Criar a Tabela de Dimensão do Cliente (Customer Dimension)
customer_dimension = df[['Customer_ID', 'Customer_Name', 'Customer_Age', 'Gender', 'Payment_Method', 'Churn', 'Returns']].drop_duplicates().reset_index(drop=True)

print("Tabela de Dimensão do Cliente (Customer Dimension):")
print(customer_dimension.head())

# Criar a Tabela de Dimensão da Categoria (Category Dimension)
category_dimension = df[['Product_Category', 'Product_Price']].drop_duplicates().reset_index(drop=True)
category_dimension['product_category_id'] = category_dimension.index + 1  # Criar IDs únicos para as categorias

print("Tabela de Dimensão da Categoria (Category Dimension):")
print(category_dimension.head())

# Mapear 'Product Category' para 'Product Category ID' na tabela principal
df = pd.merge(df, category_dimension, on=['Product_Category', 'Product_Price'])

# Criar a Tabela de Fatos (Fact Table)
fact_table = df[['Customer_ID', 'Purchase_Date', 'product_category_id', 'Quantity', 'Total_Purchase_Amount']]

print("Tabela de Fatos (Fact Table):")
print(fact_table.head())

Tabela de Dimensão do Tempo (Time Dimension):
  Purchase_Date  Day  Month  Year
0    2023-05-03    3      5  2023
1    2021-05-16   16      5  2021
2    2020-07-13   13      7  2020
3    2023-01-17   17      1  2023
4    2021-05-01    1      5  2021
Tabela de Dimensão do Cliente (Customer Dimension):
   Customer_ID   Customer_Name  Customer_Age  Gender Payment_Method  Churn  \
0        44605     John Rivera            31  Female         PayPal      0   
1        44605     John Rivera            31  Female    Credit Card      0   
2        44605     John Rivera            31  Female           Cash      0   
3        13738  Lauren Johnson            27  Female    Credit Card      0   
4        13738  Lauren Johnson            27  Female    Credit Card      0   

   Returns  
0      1.0  
1      1.0  
2      0.0  
3      1.0  
4      0.0  
Tabela de Dimensão da Categoria (Category Dimension):
  Product_Category  Product_Price  product_category_id
0             Home            177         

**Salvar Tabelas**

In [17]:
# Opcional: Salvar as tabelas criadas para arquivos CSV
time_dimension.to_csv('time_dimension.csv', index=False)
customer_dimension.to_csv('customer_dimension.csv', index=False)
category_dimension.to_csv('category_dimension.csv', index=False)
fact_table.to_csv('fact_table.csv', index=False)

# Baixar os arquivos CSV
files.download('time_dimension.csv')
files.download('customer_dimension.csv')
files.download('category_dimension.csv')
files.download('fact_table.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>